In [ ]:
#Install with: pip install PySimpleGUI
import PySimpleGUI as sg

This notebook shows how to connect TINC to an external GUI toolkit, in this case PySimpleGUI

In [ ]:
from tinc import *

Create 2 parameters for input and output

In [ ]:
in_param = Parameter("input")
out_param = Parameter("output")

Create the layout for the window:

In [ ]:
layout = [[sg.Slider(range=(1,500),
         default_value=222,
         enable_events = True,
         size=(20,15),
         orientation='horizontal',
         font=('Helvetica', 12)),
         sg.Slider(range=(1,5000),
         default_value=222,
         enable_events = True,
         size=(20,15),
         orientation='horizontal',
         font=('Helvetica', 12))],
         [sg.Button('Trigger',size=(20,4))],
         [sg.Text("Hello", size=(70,2))]]

Define and register a function that will be used as callback for input. This sets the output parameter to twice the value of the input:

In [ ]:
def compute(value):
    out_param.value = value*2

In [ ]:
in_param.register_callback(compute)

Create a Trigger parameter with a callback function to connect to the button:

In [ ]:
def updateGUI(value):
    layout[0][1].Update(value=value)
    layout[2][0].Update("Slider moved")

In [ ]:
out_param.register_callback(updateGUI)

In [ ]:
trigger = Trigger("trigger")
def trigger_compute(value):
    layout[2][0].Update("Triggered")

trigger.register_callback(trigger_compute)

Create window and run event loop:

In [ ]:
# Create the window
window = sg.Window("Demo", layout)

# Create an event loop
while True:
    event, values = window.read()
    # End program if user closes window or
    # presses the OK button
    if event == 0:
        in_param.value = values[0]    
    elif event == "Trigger":
        trigger.trigger()
    elif event == "OK" or event == sg.WIN_CLOSED:
        break

window.close()

## Interrupting computation

You can use a GUI to interrupt computation that would otherwise hang the python engine.

In order to make a callback interruptible, it must occasionally check for a global flag that you can set to exit computation.

In [ ]:
import time
interrupt = False
def interruptible_callback(value):
    global interrupt
    interrupt = False
    # Function will take around 100 seconds to complete
    for i in range(100):
        if interrupt:
            print("Forced stop")
            break
        time.sleep(1)
        print("processed ", i) 

In [ ]:
in_param.register_callback(interruptible_callback)

In [ ]:
out_param.clear_callbacks()

In [ ]:
layout = [[sg.Slider(range=(1,500),
         default_value=222,
         enable_events = True,
         size=(20,15),
         orientation='horizontal',
         font=('Helvetica', 12))],
         [sg.Button('Interrupt',size=(20,4))]]

# Create the window
window = sg.Window("Interruptible compute", layout)

import threading

def set_from_slider():
    in_param.value = values[0]
    
# Create an event loop
while True:
    event, values = window.read()
    # End program if user closes window or
    # presses the OK button
    if event == 0:
        # Because the callback is will block when setting the value we can offload the callback to a 
        # separate thread.
        # This is a similar case to receiving a change through TincClient, as the change occurs on the
        # network thread.
        x = threading.Thread(target=set_from_slider)
        x.daemon = True # Have thread destroy itself when exited.
        x.start()
    elif event == "Interrupt":
        interrupt = True
    elif event == "OK" or event == sg.WIN_CLOSED:
        break

window.close()

In [ ]:
window.close()

In [ ]:
interrupt = True